In [5]:
import torch, torchvision
import torch.optim as optim
import torch.nn as nn
import os
#import oil.augLayers as augLayers
from oil.model_trainers.classifier import Classifier
from oil.datasetup.datasets import CIFAR10, C10augLayers
from oil.datasetup.dataloaders import getUnlabLoader, getLabLoader
from oil.architectures.networkparts import layer13,ConvSmallNWN
from oil.utils.utils import cosLr, loader_to
from oil.utils.optim import AutoSWA
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## CNN Configuration

In [6]:
train_epochs = 100
net_config =        {'numClasses':10}
loader_config =     {'amnt_dev':5000,'lab_BS':50,'dataseed':0,'num_workers':4}
opt_config =        {'lr':.1, 'momentum':.9, 'weight_decay':1e-4, 'nesterov':True}
sched_config =      {'cycle_length':train_epochs,'cycle_mult':1}
trainer_config =    {}

trainer_config['log_dir'] = os.path.expanduser('~/tb-experiments/baselineSWA/')
trainer_config['description'] = 'Test being made'

def makeTrainer():
    device = torch.device('cuda')
    CNN = layer13(**net_config).to(device)
    fullCNN = nn.Sequential(C10augLayers(),CNN)
    trainset, testset = CIFAR10(False, '~/datasets/cifar10/')

    dataloaders = {}
    dataloaders['train'], dataloaders['dev'] = getLabLoader(trainset,**loader_config)
    dataloaders = {k: loader_to(device)(v) for k,v in dataloaders.items()}

    opt_constr = lambda params: AutoSWA(optim.SGD(params, **opt_config))
    lr_sched = cosLr(**sched_config)
    return Classifier(fullCNN, dataloaders, opt_constr, lr_sched, **trainer_config)


In [ ]:
trainer = makeTrainer()
trainer.train(train_epochs)
trainer.save_checkpoint()

Files already downloaded and verified
Files already downloaded and verified
Creating Train, Dev split         with 45000 Train and 5000 Dev


model: Sequential(
  (0): Sequential(
    (0): RandomTranslate(max_trans=4)
    (1): RandomHorizontalFlip()
  )
  (1): layer13(
    (activation): LeakyReLU(negative_slope=0.1)
    (conv1a): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1a): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1b): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1b): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1c): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1c): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (drop1): Dropout(p=0.5)
    (conv2a): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2a): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2b)

In [13]:
trainer.optimizer.swap_swa_sgd()
trainer.optimizer.bn_update(trainer.dataloaders['train'],trainer.model)
trainer.logStuff(2008)

        Dev_Acc  Minibatch_Loss        lr
92009  0.940239        0.052646  0.000746
